## TOC:
* [Abstract](#abs)
* [Introduction](#intro)
* [Exact Riemann Solver](#ers)
    * [Riemann Problem](#ers-rp)
    * [Solution of Riemann Problem](#ers-so)
    * [Tests](#ers-te)
        * [Sod (shock tube) test](#ers-te-so)
        * [123 test](#ers-te-123)
        * [Left blast wave test](#ers-te-lbw)
        * [Right blast wave test](#ers-te-rbw)
        * [Two shocks test](#ers-te-tw)
    * [Data Visualisation](#ers-dv)
* [Machine Learning Powered Riemann Solver](#mlrs)
    * [Range of Interest](#mlrs-ri)
    * [Data](#mlrs-data)
    * [Models](#mlrs-mo)
    * [Tests](#mlrs-te)
* [Conclusion](#con)
* [References](#refer)
* [Similar Works](#similar-works)
* [Appendix I: Derivation of pressure function for shocks and rarefactions](#app-1)

## Abstract <a class="anchor" id="abs"></a>

## Introduction <a class="anchor" id="intro"></a>

[Godunov 1959](#ref-godunov-1959) introduced a conservative scheme for numerically solving non-linear systems of hyperbolic conervation laws. His original 1st-order scheme three main steps:
* Data reconstruction
* Solving local Riemann problem at cell boundaries
* Evolving hydro variables

Since 1959, higher-order schemes were invented by varius people based on the Godunov scheme (references).

Briefly mention higher-order TVD schemes:
* [ ] Wighted average flux (WAF) schemes
* [ ] MUSCL-Hancock scheme

The key ingredient of all these schemes is the solution of local Riemann problem.

Briefly mention and reference different Riemann solvers here:
* [ ] HLL and HLLC solvers
* [ ] Roe solver
* [ ] Approximate-state solvers
* [ ] Exact solver (more information in a separate chapter)

In a typical hydrodynamical simulations, Riemann problem must be solved at all cell boundaries for every time step, thus any improvement in the performance of this task will lead to a significant improvement in overall performance of hydrodynamical simulations.

The purpose of this study is to introduce an approximate Machine Learning based Riemann Solver (MLRS hearby) with O(1) complexity to be substitute by current solvers. Our method consist of a multi-layer netwrork trained on a sample of Riemann problem solutions generated by the exact Riemann solver within a wide range of hydrodynamical varibales. Since we are specifically interested in the range of variables relevant for astrophisycal simulations, we limit our scope of interest to only ideal gasses and the solution of the Riemann problem at its initial boundary.

## Exact Riemann Solver <a class="anchor" id="ers"></a>

Riemann problem does not have a closed-form solution and what we refer here as the exact Riemann solver is an iterative scheme that can be used to calculate the solution of a Riemann problem with a desired accuracy. Here, we are interested in solution of Riemann problem for ideal gasses at the initial boundary of two states (look at [Colella and Glaz 1985](#ref-colella-1985) for a discussion about general Riemann solvers).

The exact solution of the Riemann problem is an important reference solution to be used to asses the accuracy and performance of numerical simulations and methods. It is also a good example of a simple intial condition which leads to a rich non-trivial result containing the mathematical and physical essensce of the conservation laws.

The solution of the Riemann problem can be seen as the non-linear superposition of solutions of local Riemann problems [Glimm 1965](#ref-glimm-1965).

In this chapter we present a procedure to iteratively solve the Riemann problem of inviscid compressible flow.

### Riemann Problem <a class="anchor" id="ers-rp"></a>

A Riemann problem is an Initial Value Problem (IVP) composed of conservation laws together with intiial piecewise constant data having a discontinuity at the centre.

In Cartesian space, conservation laws can be described as,

$$\partial_t U + \partial_x F(U) = 0$$

where $U$ is a vector of conservative variable and $F(U)$ is their fluxes. In three-dimensional space $U$ can be written as,

$$U = \begin{bmatrix} \rho \\ \rho u \\ \rho v \\ \rho w \\ E\end{bmatrix}$$

where $u$, $v$ and $w$ are velocities in $x$, $y$ and $z$ directions and $E$ is the total energy consists of kinetic and internal energies and it equals to $\frac{1}{2} \rho \mathbf{v}^2 + e_{\text{int}}$. The corresponding flux of the flow in $x$ direction can be written as:

$$F_x(U) = \begin{bmatrix} \rho u \\ \rho u^2 + p \\ \rho u v \\ \rho u w \\ u(E + p) \end{bmatrix}$$

where $p$ is the pressure and can be calculated by the caloric ideal gas Equation of State (EoS):

$$e_{\text{int}} = \frac{p}{(\gamma - 1) \rho}$$

The initial piecewise constant condition is also written as,

$$U(x,t=0) = U^{(0)}(x) = \begin{cases}
U_L & \text{if} \quad x < 0 \\
U_R & \text{if} \quad x > 0
\end{cases}$$

Where $U_L$ and $U_R$ are vectors of conservative variables and have constant values over the domain of interest.

Here, we assume that the system is strictly hyperbolic, meaning the
eigenvalues of the Jacobian matrix $\mathbf{A} = \partial_{u_j} f_i$ are
real and distinct,

$$\lambda_1 < \lambda_2 < ... < \lambda_m$$

Each eigenvector corresponds to a wave emerging from the discontinuity and propagating with the velocity equals to its eigenvalue. Generally, the solution of the Riemann problem consists of three types of waves,

* one linearly degenerate wave (the contact discontinuity)
* (nonlinear) Shock waves
* (nonlinear) rarefaction (fan) waves

Contact discontinuities are surfaces that separate zones of different density and temperature at fixed pressure. By definition such a surface is in pressure equilibrium, and no gas flows across it. Usually, when the tangential components of velocity on one side differs considerably from that of the gas on the other side, the surface is called a slip discontinuity. The boundary of a supersonic jet and the ambient gas is an example of a slip discontinuity.

The two types of nonlinear waves (shocks and rarefactions) arise from abrupt changes in pressure. Shock fronts accompany compression of the medium and rarefactions accompany expansion of the medium.

These waves are separating four different states where the conservative vector $U$ acquires from the left to the right the following values, $U_L$, $U_{*L}$, $U_{*R}$ and $U_R$. The symbol $*$ identify points located in the star region between the nonlinear waves.

### Solution of Riemann Problem <a class="anchor" id="ers-so"></a>

The solution for the pressure at contact discontinuity of the Riemann problem for an ideal gas, $p_*$, is given by the root of the following equation, called the **pressure function**,

$$
f(p, W_L, W_R) \equiv f_L(p,W_L) + f_R(p, W_R) + \Delta u = 0,
\quad \Delta u \equiv u_R - u_L
$$

where \$f_K$ for $k \in [L, R]\$ represents relations across the left and the right nonlinear waves and is given by:

$$
f_K \left(p, W_K\right) = \begin{cases}
  (p - p_K) \sqrt{(\frac{A_K}{p + B_K})}
  & \text{if}\quad p > p_K \quad \text{shock} \\
  & \\
  \frac{2 c_{sK}}{\gamma - 1} \left[
    \left( \frac{p}{p_K} \right)^{\frac{\gamma - 1}{2 \gamma}} - 1
  \right]
  & \text{if}\quad p < p_K \quad \text{rarefaction}
\end{cases}
$$

$$
A_K = \frac{2}{(\gamma + 1) \rho_K}, \quad
B_K = \frac{\gamma - 1}{\gamma + 1} p_K
$$

Following the calculation of $p_*$, the velocity at contact discontinuity, $u_*$, which is a function of the left and right velocities and $p_*$, can be calculated as,

$$u_* = \frac 1 2 (u_L + u_R) + \frac 1 2 \left[ f_R(p_*) - f_L(p_*) \right]$$

The remaining unknowns can be found by using standard gas dynamics relations.

In order to investigate the behaviour of the pressure function at a given data ($u_L = (\rho_L, u_L, v_L, w_L, p_L)^T$ and $u_R = (\rho_R, u_R, v_R, w_R, p_R)^T$), we need to calculate the first and the second derivative of the function w.r.t $p$. The first derivative of the pressure function can be written as,

$$
f^{\prime}_K =
\begin{cases}
  (\frac{A_K}{p + B_K})^{1/2}
  \left(
    1 - \frac{p - p_K}{2 (B_k + p)}
  \right) & \text{if}\quad p > p_K \quad \text{shock} \\
  & \\
  \frac{1}{\rho_K c_s^K}
  \left(
    \frac{p}{p_K}
  \right)^{\frac{- (\gamma + 1)}{2 \gamma}}
  & \text{if}\quad p \lt p_K \quad \text{rarefaction}
\end{cases}
$$

as $f^\prime = f_L^\prime + f_R^\prime$ and both $f_{L,R}^\prime > 0$, we can see that the pressure function is monotonic function.

Considering the second derivative of the pressure function

$$
f^{\prime\prime}_K =
\begin{cases}
\frac 1 4 \left( \frac{ A_K }{ p + B_K } \right)^{1/2}
\left[
  \frac{ 4B_K + 3p + p_K }{ ( B_K + p )^2 }
\right]
& \text{if}\quad p > p_K \quad \text{shock} \\
& \\
- \frac{ (\gamma - 1) c_{sK} }{ 2 \gamma^2 p_K^2 }
\left(
  \frac{ p }{ p_K }
\right)^{ - ( 3\gamma + 1 ) / 2\gamma }
& \text{if}\quad p \leq p_K \quad \text{rarefaction}
\end{cases}
$$

since $f^{\prime\prime} = f_l^{\prime\prime} + f_r^{\prime\prime}$ and both $f_{L,R}^{\prime\prime} < 0$, the pressure function is downward concave.

The following figure shows the behaviour of the pressure function in the solution of the Riemann problem,

![pressure-function](assets/pressure-function.png)

In the above figure, we define,

$$
\begin{aligned}
  & p_{\text{min}} = \text{min}(p_L, p_R) \\
  & p_{\text{max}} = \text{max}(p_L, p_R) \\
  & f_{\text{min}} = f(p_{\text{min}}) \\
  & f_{\text{max}} = f(p_{\text{max}})
\end{aligned}
$$

For a given $p_L$ and $p_R$, it is the velocity differece of two sides, $\Delta u$, that determines the value of $p_*$.

Three distinct intervals, $I_1$, $I_2$ and $I_3$, can be identified in the domain of the pressure function:

$$
\begin{aligned}
  & p_*\quad\text{lies in}\quad I_1 = (0, p_{\text{min}})\quad
  &\text{if}\quad f_{\text{min}} > 0\quad\text{and}\quad f_{\text{max}} > 0\\
  & p_*\quad\text{lies in}\quad I_2 = [p_{\text{min}}, p_{\text{max}}]\quad
  &\text{if}\quad f_{\text{min}} \leq 0\quad\text{and}\quad f_{\text{max}} \geq 0\\
  & p_*\quad\text{lies in}\quad I_3 = (p_{\text{max}}, \infty)\quad
  &\text{if}\quad f_{\text{min}} < 0\quad\text{and}\quad f_{\text{max}} < 0
\end{aligned}
$$

For suficiently large $\Delta u$ as in $(\Delta u)_1$, the value of $p_*$ lies in the $I_1$ region and since the root of the pressure function is less than $p_L$ and $p_R$, we end up having two rarefactions. We can follow the same logic to explain $(\Delta u)_2$ and $(\Delta u)_3$.

Note for a positive solution of star region pressure $p_*$ we require $f(0) < 0$. This leads to the pressure positivity condition,

$$
(\Delta u)_{\text{crit}}
  \equiv \frac{2 c_{sL}}{\gamma -1} + \frac{2 c_{sR}}{\gamma -1}
  > u_R - u_L
$$

Note that vacuum is created by the nonlinear waves if this condition is violated.

Add a new subsection for the solution of Riemann problem in the presense of vacuum:
* at the right side
* at the left side
* at both sides

### Tests <a class="anchor" id="ers-te"></a>

#### Sod (shock tube) test <a class="anchor" id="ers-te-so"></a>

#### 123 test <a class="anchor" id="ers-te-123"></a>

#### Left blast wave test <a class="anchor" id="ers-te-lbw"></a>

#### Rigth blast wave test <a class="anchor" id="ers-te-rbw"></a>

#### Two shocks test <a class="anchor" id="ers-te-tw"></a>

## Machine Learning Powered Riemann Solver <a class="anchor" id="mlrs"></a>

### Range of Interest <a class="anchor" id="mlrs-ri"></a>

In [ ]:
# global packages and variables
from ml_riemann_solver import MLRiemannSolver as MLRS

from math import sqrt

import os
from pathlib import Path
import pickle
import numpy as np
import astropy.units as U
import astropy.constants as C
import csv

# !pip install plotly
import plotly
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots
pio.templates.default = "simple_white"

# Backup directory
BACKUP_DIRECTORY = os.environ['HOME'] + '/.mlrs'
Path(BACKUP_DIRECTORY).mkdir(parents=True, exist_ok=True)

rho_min=1e-4 * C.m_p * U.cm**-3
rho_max=1e2 * C.m_p * U.cm**-3
p_min=1e-16 * U.N / U.m**2
p_max=1e-12 * U.N / U.m**2
v_min=-150 * U.km / U.s
v_max=150 * U.km / U.s

def get_filepath(rho_min, rho_max, rho_len, p_min, p_max, p_len, v_min, v_max, v_len, G):
    f_rho = f"{rho_min.si.value:4.3e}-{rho_max.si.value:4.3e}{rho_min.si.unit:FITS}-{rho_len}"
    f_p = f"{p_min.si.value:4.3e}-{p_max.si.value:4.3e}{p_min.si.unit:FITS}-{p_len}"
    f_v = f"{v_min.si.value:4.3e}-{v_max.si.value:4.3e}{v_min.si.unit:FITS}-{v_len}"
    
    return f"{BACKUP_DIRECTORY}/{f_rho}__{f_p}__{f_v}_G{G:3.2f}.csv".replace(' ', '_')

### Data <a class="anchor" id="mlrs-data"></a>

In [ ]:
def guess_p_star(rho_l, v_l, p_l, cs_l, rho_r, v_r, p_r, cs_r, G=5./3.):
    G1 = (G - 1.0)/(2.0*G)
    G2 = (G + 1.0)/(2.0*G)
    G3 = 2.0*G/(G - 1.0)
    G4 = 2.0/(G - 1.0)
    G5 = 2.0/(G + 1.0)
    G6 = (G - 1.0)/(G + 1.0)
    G7 = (G - 1.0)/2.0
    G8 = G - 1.0
    
    q_user = 2.0
    
    cup = 0.25 * (rho_l + rho_r) * (cs_l + cs_r)
    ppv = max(0.5 * (p_l + p_r) + 0.5 * (v_l - v_r) * cup, 0.0)
    p_min = min(p_l, p_r)
    p_max = max(p_l, p_r)
    q_max = p_max / p_min

    if q_max <= q_user and (p_min <= ppv and ppv <= p_max):
        # PVRS Riemann solver
        pm = ppv
    else:
        if ppv <= p_min:
            # Two-Rarefaction Riemann solver
            pq = (p_l / p_r)**G1
            um = (pq * v_l / cs_l + v_r / cs_r + G4 * (pq - 1.0)) / (pq / cs_l + 1.0 / cs_r)
            ptl = 1.0 + G7 * (v_l - um) / cs_l
            ptr = 1.0 + G7 * (um - v_r) / cs_r
            pm = 0.5 * (p_l * ptl**G3 + p_r * ptr**G3)            
        else:
            # Two-Shock Riemann solver
            gel = sqrt((G5 / rho_l) / (G6 * p_l + ppv))
            ger = sqrt((G5 / rho_r) / (G6 * p_r + ppv))
            pm = (gel * p_l + ger * p_r - (v_r - v_l)) / (gel + ger)
    
    return pm

def wave_function(rho, v, p, p_s, G=5./3.):
    if rho <= 0.0 or p <= 0.0:
        print('wave function, negative values', rho, v, p, p_s, G)

    if p_s > p:
        Ak = 2 / ((G+1) * rho)
        Bk = (G-1) / (G+1) * p
        factor = sqrt(Ak / (Bk + p_s))
        f = factor * (p_s - p)p_star = p_prev - (f_l + f_r + (v_r - v_l)) / (fprime_l + fprime_r)
        fprime = factor * (1 - (p_s - p) / (2 * (Bk + p_s)))
    else:
        cs = sqrt(G * p / rho)
        f = 2 * cs / (G-1) * ((p_s / p)**((G-1) / (2*G)) - 1)
        fprime = 1. / (rho * cs) * (p_s / p)**(-(G+1) / (2*G))

    return f, fprime


def iterate(rho_l, v_l, p_l, rho_r, v_r, p_r, G=5./3., tolerance=1e-10):
    """
    :param l: left side primitive variables (rho, v, p)
    :param r: right side primitive variables (rho, v, p)
    
    :return p_star
    """
    cs_l = sqrt(G * p_l / rho_l)
    cs_r = sqrt(G * p_r / rho_r)
    
    guessed_p = abs(guess_p_star(rho_l, v_l, p_l, cs_l, rho_r, v_r, p_r, cs_r, G=G))
    
    found = False

    p_star = guessed_p  
    p_prev = p_star
    
    for i in range(10000):
        f_l, fprime_l = wave_function(rho_l, v_l, p_l, p_star, G=G)
        f_r, fprime_r = wave_function(rho_r, v_r, p_r, p_star, G=G)

        p_star = p_prev - (f_l + f_r + (v_r - v_l)) / (fprime_l + fprime_r)

        if 2 * abs((p_star - p_prev) / (p_star + p_prev)) < tolerance:
            found = True
            break

#         if p_star < 0:
#             p_star = abs(p_star * tolerance)

        p_prev = p_star
            
    if not found:
#         print(f'Not found! ({check}) guess: {guessed_p}, p_star: {p_star}, p: ({p_l}, {p_r}), v: ({v_l}, {v_r}), rho: ({rho_l}, {rho_r})')
        return None
#     else:
#         print(f'Found! guess: {guessed_p}, p_star: {p_star}, p: ({p_l}, {p_r}), v: ({v_l}, {v_r}), rho: ({rho_l}, {rho_r})')
        
    return p_star

In [ ]:
def write_data(rho_min, rho_max, rho_len, p_min, p_max, p_len, v_min, v_max, v_len, G, rewrite=False):
    filepath = get_filepath(rho_min, rho_max, rho_len, p_min, p_max, p_len, v_min, v_max, v_len, G)
    print(filepath)

    if Path(filepath).is_file() and not rewrite:
        print("CSV file exist")
        return
    
    ranges = {
        'rho': np.logspace(np.log10(rho_min.si.value), np.log10(rho_max.si.value), rho_len),
        'p': np.logspace(np.log10(p_min.si.value), np.log10(p_max.si.value), p_len,),
        'v': np.linspace(v_min.si.value, v_max.si.value, v_len,),
    }
    
    fileds = [
        f'rho_l [{rho_min.si.unit:FITS}]', f'v_l [{v_min.si.unit:FITS}]', f'p_l [{p_min.si.unit:FITS}]',
        f'rho_r [{rho_min.si.unit:FITS}]', f'v_r [{v_min.si.unit:FITS}]', f'p_r [{p_min.si.unit:FITS}]',
        f'p_star [{p_min.si.unit:FITS}]'
    ]
    
    with open(filepath, 'w') as csvfile:  
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(fileds)
        
        print('Start...')
        
        total = rho_len**2 * v_len**2 * p_len**2
        cntr = 0
        percent = 0
        
        print(f'{percent/10000.0:7.4f}%', end='\r')
        
        for rho_l in ranges['rho']:
            for rho_r in ranges['rho']:
                for p_l in ranges['p']:
                    for p_r in ranges['p']:
                        for v_l in ranges['v']:
                            for v_r in ranges['v']:
                                cs_l = sqrt(G * p_l / rho_l)
                                cs_r = sqrt(G * p_r / rho_r)
                                
                                # pressure positivity test
                                if (2. / (G - 1)) * (cs_l + cs_r) <= (v_r - v_l):
                                    p_star = 0.0
                                else:
                                    p_star = iterate(rho_l, v_l, p_l, rho_r, v_r, p_r, G=G)
                                
                                if p_star is not None and p_star > 0:
                                    csvwriter.writerow([rho_l, v_l, p_l, rho_r, v_r, p_r, p_star])
                                
                                cntr += 1
                                new_percent = int(cntr / total * 1000000)
                                if new_percent > percent:
                                    percent = new_percent
                                    print(f'{percent/10000.0:7.4f}%', end='\r')
    
# write_data(
#     rho_min=rho_min, rho_max=rho_max, rho_len=10,
#     p_min=p_min, p_max=p_max, p_len=10,
#     v_min=v_min, v_max=v_max, v_len=10,
#     G=5./3.,
#     rewrite=True
# )

write_data(
    p_min=0.001 * U.N / U.m**2, p_max=1000 * U.N / U.m**2, p_len=7,
    v_min=-1 * U.m / U.s, v_max=1 * U.m / U.s, v_len=5,
    rho_min=0.01 * U.kg / U.m**2, rho_max=100 * U.kg / U.m**2, rho_len=5,
    G=5.0/3.0,
    rewrite=True
)

### Data Visualisation <a class="anchor" id="ers-dv"></a>

In [ ]:
def visualize_data(rho_min, rho_max, rho_len, p_min, p_max, p_len, v_min, v_max, v_len, G, canvas_size=(256, 256)):
    fpath = get_filepath(
        rho_min=1e-4 * C.m_p * U.cm**-3, rho_max=1e2 * C.m_p * U.cm**-3, rho_len=10,
        p_min=1e-16 * U.N / U.m**2, p_max=1e-12 * U.N / U.m**2, p_len=10,
        v_min=-150 * U.km / U.s, v_max=150 * U.km / U.s, v_len=10,
        G=5./3.,
    )
    
    if not Path(fpath).is_file():
        print('Not found!')
        return
    
    rcanvas = np.zeros(canvas_size)
    rcounts = np.zeros(canvas_size, dtype=np.int)
    vcanvas = np.zeros(canvas_size)
    vcounts = np.zeros(canvas_size, dtype=np.int)
    pcanvas = np.zeros(canvas_size)
    pcounts = np.zeros(canvas_size, dtype=np.int)
    
    f = open(fpath, 'r')
    csv_reader = csv.reader(f)
    next(csv_reader)
    
    rmin, rmax = rho_min.si.value, rho_max.si.value
    pmin, pmax = p_min.si.value, p_max.si.value
    vmin, vmax = v_min.si.value, v_max.si.value

    for irow, row in enumerate(csv_reader):
        rl, vl, pl = float(row[0]), float(row[1]), float(row[2])
        rr, vr, pr = float(row[3]), float(row[4]), float(row[5])
        p_star = float(row[6])
        
        if p_star is None:
            print('p_star is none')
            continue
        
        ir = min(max(0, int((np.log10(rl/rmin)) / (np.log10(rmax/rmin)) * canvas_size[0])), canvas_size[0]-1)
        jr = min(max(0, int((np.log10(rr/rmin)) / (np.log10(rmax/rmin)) * canvas_size[1])), canvas_size[1]-1)
        
        iv = min(max(0, int((vl-vmin) / (vmax-vmin) * canvas_size[0])), canvas_size[0]-1)
        jv = min(max(0, int((vr-vmin) / (vmax-vmin) * canvas_size[1])), canvas_size[1]-1)
        
        ip = min(max(0, int((np.log10(pl/pmin)) / (np.log10(pmax/pmin)) * canvas_size[0])), canvas_size[0]-1)
        jp = min(max(0, int((np.log10(pr/pmin)) / (np.log10(pmax/pmin)) * canvas_size[1])), canvas_size[1]-1)

        rcanvas[ir, jr] = (rcounts[ir, jr] * rcanvas[ir, jr] + p_star) / (rcounts[ir, jr] + 1)
        rcounts[ir, jr] += 1
        
        vcanvas[iv, jv] = (vcounts[iv, jv] * vcanvas[iv, jv] + p_star) / (vcounts[iv, jv] + 1)
        vcounts[iv, jv] += 1
        
        pcanvas[ip, jp] = (pcounts[ip, jp] * pcanvas[ip, jp] + p_star) / (pcounts[ip, jp] + 1)
        pcounts[ip, jp] += 1

    f.close()
        
    return rcanvas, rcounts, vcanvas, vcounts, pcanvas, pcounts
        
rcanvas, rcounts, vcanvas, vcounts, pcanvas, pcounts = visualize_data(
    rho_min=rho_min, rho_max=rho_max, rho_len=10,
    p_min=p_min, p_max=p_max, p_len=10,
    v_min=v_min, v_max=v_max, v_len=10,
    G=5./3.,
    canvas_size=(10, 10),
)

In [ ]:
def plot_heatmap(data, title):
    fig = go.Figure()
    fig.add_trace(go.Heatmap(
        z=data, colorscale='Spectral', reversescale=True,
    ))
    fig.update_layout(
        title=title, width=500, height=500,
        xaxis=dict(mirror=True),
        yaxis=dict(mirror=True),
    )
    fig.show()

plot_heatmap(np.log10(rcanvas), 'rho')
plot_heatmap(np.log10(vcanvas), 'v')
plot_heatmap(np.log10(pcanvas), 'p')

### Models <a class="anchor" id="mlrs-mo"></a>

In [ ]:
# !pip3 install torch torchvision

from collections import OrderedDict
import torch
from torch import from_numpy, optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
from torchvision.utils import make_grid
import h5py

In [ ]:
from math import log10

def load_data(path):
    if not Path(path).is_file():
        print("File does not exist!")
        return None
    
    result = np.genfromtxt(path, skip_header=1, delimiter=',', converters={
        0: lambda s: float(s),
        1: lambda s: float(s),
        2: lambda s: float(s),
        3: lambda s: float(s),
        4: lambda s: float(s),
        5: lambda s: float(s),
        6: lambda s: float(s),
    })
    
    return result

fpath = get_filepath(
    rho_min=rho_min, rho_max=rho_max, rho_len=10,
    p_min=p_min, p_max=p_max, p_len=10,
    v_min=v_min, v_max=v_max, v_len=10,
    G=5./3.,
)

M = load_data(fpath)
M = M[np.isfinite(M).all(axis=1)]
M = M[M[:, 6] > 0]

print(len(M))

In [ ]:
data = np.zeros((len(M), 6))
filt = np.zeros((len(M), 6))

drho = (np.log10(rho_max.si.value * rho_min.si.value)) / 2
dp = np.log10(p_max.si.value * p_min.si.value) / 2

normalized='norm'
data[:, 0] = (np.log10(M[:, 0]) - drho) / drho
data[:, 1] = (np.log10(M[:, 2]) - dp) / dp
data[:, 2] = (np.log10(M[:, 3]) - drho) / drho
data[:, 3] = (np.log10(M[:, 5]) - dp) / dp
data[:, 4] = (M[:, 4] - M[:, 1]) / (2 * v_max.si.value)
data[:, 5] = (np.log10(M[:, 6]) - dp) / dp

# normalized='not_norm'
# data[:, 0] = M[:, 0]
# data[:, 1] = M[:, 2]
# data[:, 2] = M[:, 3]
# data[:, 3] = M[:, 5]
# data[:, 4] = M[:, 4] - M[:, 1]
# data[:, 5] = M[:, 6]

data = data[~np.isnan(data).any(axis=1) ]

filt[:, 0] = M[:, 0]
filt[:, 1] = M[:, 2]
filt[:, 2] = M[:, 3]
filt[:, 3] = M[:, 5]
filt[:, 4] = M[:, 4] - M[:, 1]
filt[:, 5] = M[:, 6]

mask = (filt[:, 0] > 0) & (filt[:, 1] > 0) & (filt[:, 2] > 0) & (filt[:, 3] > 0) & (filt[:, 5] > 0)

data = data[mask]

dataset = from_numpy(data)
print(f"Dataset shape: {dataset.size()}")
print(f"first row:  {[float(x) for x in dataset[0]]}")
print(f"middle row: {[float(x) for x in dataset[int(len(dataset) / 2)]]}")
print(f"last row:   {[float(x) for x in dataset[-1]]}")

In [ ]:
import numpy as np
import time

class RSModel(nn.Module): 
    ACTIVATION = {
        'ReLu': nn.ReLU(),
        'LogSigmoid': nn.LogSigmoid(),
        'Sigmoid': nn.Sigmoid(),
        'Tanh': nn.Tanh(),
        'LeakyReLu': nn.LeakyReLU(),
        'Softsign': nn.Softsign(),
    }

    LOSS = {
        'L1': nn.L1Loss(),
        'MSE': nn.MSELoss(),
        'KLDiv': nn.KLDivLoss(reduction = 'batchmean'),
    }

    OPTIMIZER = {
        'Adadelta': lambda x, y: optim.Adadelta(x, lr=y, rho=0.9, eps=1e-06, weight_decay=0),
        'Adagrad': lambda x, y: optim.Adagrad(x, lr=y, lr_decay=0, weight_decay=0, initial_accumulator_value=0, eps=1e-10),
        'Adam': lambda x, y: optim.Adam(x, lr=y),
        'AdamW': lambda x, y: optim.AdamW(x, lr=y, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01, amsgrad=False),
        'LBFGS': lambda x, y: optim.LBFGS(x, lr=y, max_iter=20, max_eval=None, tolerance_grad=1e-07, tolerance_change=1e-09, history_size=100, line_search_fn=None),
        'RMSprop': lambda x, y: optim.RMSprop(x, lr=y, alpha=0.99, eps=1e-08, weight_decay=0, momentum=0, centered=False),
        'Rprop': lambda x, y: optim.Rprop(x, lr=y, etas=(0.5, 1.2), step_sizes=(1e-06, 50)),
        'SGD': lambda x, y: optim.SGD(x, lr=y, momentum=0.9),
    }
    
    def __init__(self, arch, loss, optimizer, learning_rate):
        """
        arch: [(5, 128), 'ReLu', (128, 128), 'Sigmoid', (128, 1)]
        """
        super().__init__()
        
        self.log = []
        
        self.arch = arch
        self.arch_str = '-'.join([str(x) for x in arch]).replace(' ', '').replace('(', '').replace(')', '').replace(',', '_')
        self.logging(f"Arch: {self.arch_str}")
        
        self.layer_sizes = [x[0] for x in arch if isinstance(x, tuple)] + [arch[-1][1]]
        self.activations_labels = [x for x in arch if isinstance(x, str)]
        
        nlayer = 0
        self.layers = []
        for l in self.arch:
            if isinstance(l, tuple):
                nlayer += 1
                self.layers.append((f"L{nlayer}", nn.Linear(l[0], l[1])))
            elif isinstance(l, str):
                self.layers.append((l, self.ACTIVATION[l]))
                
        self.model = nn.Sequential(OrderedDict(self.layers))
            
        self.loss_name = loss
        self.logging(f"Loss function: {loss}")
        self.loss = self.LOSS[loss]
            
        self.optimizer_name = optimizer
        self.learning_rate = learning_rate
        self.logging(f"Optimizaer: {optimizer}, learning rate: {learning_rate}")
        
        self.losses = []
        self.loss_slopes = []
        
    def logging(self, msg):
        self.log.append(msg)
        print(msg)
        
    def load(self, dataset, validation_fraction, batch_size):
        val_size = int(validation_fraction * len(dataset))
        train_size = len(dataset) - val_size
        
        self.logging(f"Loading: training size: {train_size} validation size: {val_size}")
        
        self.train_ds, self.val_ds = random_split(dataset, [train_size, val_size])
        
        self.train_loader = DataLoader(self.train_ds, batch_size, shuffle=True)
        self.val_loader = DataLoader(self.val_ds, batch_size) 

    def forward(self, x):
        out = self.model(x)
        return out
        
    def train(self, n_epochs, break_criterion=0.01):
        optimizer = self.OPTIMIZER[self.optimizer_name](self.parameters(), self.learning_rate)
        
        for epoch in range(n_epochs):
            losses = 0
            for i, batch in enumerate(self.train_loader):
                optimizer.zero_grad()
                
                inputs = batch[:, :self.layer_sizes[0]].type(torch.FloatTensor)
                expected = batch[:, -self.layer_sizes[-1]].type(torch.FloatTensor)
                
                outputs = self(inputs)
                loss = self.loss(torch.squeeze(outputs), expected)
                loss.backward()
                optimizer.step()
                
                losses += loss.item()
            
            self.losses.append(losses / len(self.train_loader))
            print(f"Training ({epoch + 1} / {n_epochs}): loss = {self.losses[-1]:6.4e}")
            
    def validation(self, conv_outputs, conv_expected, tolerance, do_print=(False, 5)):
        with torch.no_grad():
            n_corrects, n_total = 0, 0
            
            for i, batch in enumerate(self.val_loader):
                inputs = torch.squeeze(batch[:, :self.layer_sizes[0]].type(torch.FloatTensor))
                expected = torch.squeeze(batch[:, -self.layer_sizes[-1]].type(torch.FloatTensor))
                
                outputs = self(inputs)
                
                outputs = conv_outputs(torch.squeeze(outputs))
                expected = conv_expected(torch.squeeze(expected))
                
                n_total += len(outputs)
                n_corrects += (
                    np.abs((outputs - expected) / expected) < tolerance
                ).sum().item()
                
                
                if do_print[0] and i % int((len(self.val_loader) - 1) / do_print[1]) == 0:
                    print(f"{i:6d} - inputs: {inputs[0]}")
                    print(f"       - output: {outputs[0]}")
                    print(f"       - expected: {expected[0]}")
                    print(f"       - diff: {abs((outputs[0] - expected[0])/expected[0])}")
                
        accuracy = n_corrects / n_total
        self.logging(f"Validation: accuracy = {accuracy * 100:6.4e}%")
        return accuracy
    
    def id(self):
        return f"{self.arch_str}-{self.loss_name}-{self.optimizer_name}-{str(self.learning_rate).replace('.', '_')}"
    
    def image_path(self, prefix):
        return f"{prefix}/{self.id()}.png"
    
    def save_losses(self, prefix='.'):
        csv_path = f'{prefix}/{self.id()}.csv'
        
        with open(csv_path, 'w') as f:
            for l in self.losses:
                f.write(str(l) + '\n')
                
        self.logging(f"Saved the losses into {csv_path}")
            
    def plot(self, y_range, width=500, height=500, prefix='.'):
        fig = go.Figure()
        
        fig.add_trace(go.Scatter(
            x=list(range(len(self.losses))), y=self.losses
        ))
        
        for i, msg in enumerate(self.log):
            fig.add_annotation(
                xref='paper', yref='paper', xanchor='left', yanchor='top',
                x=0.05, y=-0.2 -i * 0.06,
                showarrow=False, text=msg,
            )
            
        caption_height = (0.2 + (len(self.log) + 1) * 0.06) * height
        
        fig.update_layout(
            width=width, height=caption_height + height, margin=dict(t=20, r=20, b=20 + caption_height, l=20),
            xaxis=dict(mirror=True, type='linear', title='# of epoch'),
            yaxis=dict(mirror=True, type='log', title=self.loss_name, range=y_range),
        )
        
        fig.write_image(self.image_path(prefix), width=width, height=caption_height + height, scale=3)
        
        fig.show()

In [ ]:
import time

print(pio.orca.status)


def brute_force(
    niter=32,
    v1=lambda x: 10**(dp * x + dp),
    v2=lambda x: 10**(dp * x + dp),
    vrange=1e-6,
    yrange=(-6, -1.5)):
    prefix = f'/Users/Saeed/Desktop/models.{normalized}'

    for act_1 in ['ReLu']: # , 'Tanh']:
#         for act_2 in ['ReLu', 'Tanh']:
        for act_3 in ['Sigmoid', 'Tanh']:
            for loss in ['MSE',]:
                for lr in [0.01, 0.001, 0.0001, 0.00001]:
                    for opt in ['Adam', 'SGD']:#, 'AdamW', 'Rprop']:
#                         model = RSModel([(5, 19), act_1, (19, 67), act_2, (67, 128), act_3, (128, 1)], loss, opt, lr)
                        model = RSModel([(5, 256), act_1, (256, 128), act_3, (128, 1)], loss, opt, lr)
                        image_path = model.image_path(prefix)
                        saved_model_path = f"{prefix}/{model.id()}.pt"

                        if Path(image_path).is_file():
                            print(f"File exists: {image_path}")
                            del model
                            continue

                        if Path(saved_model_path).is_file():
                            print(f"Model is already saved: {saved_model_path}")
                            del model
                            continue

                        model.load(dataset, 0.05, 128)
                        model.train(niter)
                        model.validation(v1, v2, vrange, do_print=(True, 5))
                        model.save_losses(prefix=prefix)
                        torch.save(model.state_dict(), saved_model_path)
                        model.plot(y_range=yrange, prefix=prefix)
                        del model
                            
def long_brute_force(
    niter=1024,
    v1=lambda x: 10**(dp * x + dp),
    v2=lambda x: 10**(dp * x + dp),
    vrange=1e-6,
    yrange=(-8, -2)):
    prefix = f'/Users/Saeed/Desktop/long_run_models.{normalized}'
    
    for act in ['Tanh', 'Sigmoid']:
        for n in [32, 64, 128, 256, 512]:
            model = RSModel([(5, n), 'ReLu', (n, n), act, (n, 1)], 'MSE', 'Adam', 0.001)
            image_path = model.image_path(prefix)
            saved_model_path = f"{prefix}/{model.id()}.pt"
            
            if Path(image_path).is_file():
                print(f"Model exists: {image_path}")
                del model
                continue
                
            if Path(saved_model_path).is_file():
                print(f"Model is already saved: {saved_model_path}")
                del model
                continue
            
            model.load(dataset, 0.05, 128)
            model.train(niter)
            model.validation(v1, v2, vrange)
            
            try:
                model.save_losses(prefix=prefix)
            except:
                time.sleep(10)
                try:
                    model.save_losses(prefix=prefix)
                except:
                    print(f"save_losses failed")
                
            try:
                torch.save(model.state_dict(), saved_model_path)
            except:
                time.sleep(10)
                try:
                    torch.save(model.state_dict(), saved_model_path)
                except:
                    print(f"model save failed")
            
            try:
                model.plot(y_range=yrange, prefix=prefix)
            except:
                try:
                    model.plot(y_range=yrange, prefix=prefix)
                except:
                    print(f"plotly failed")
            
            del model
            
def single_learning(
    niter=10240,
    lr=0.001,
    v1=lambda x: 10**(dp * x + dp),
    v2=lambda x: 10**(dp * x + dp),
    vrange=1e-6):
    
    prefix = f'/Users/Saeed/Desktop/single_long_run_models.{normalized}'
    
    n = 128
    
    model = RSModel([(5, n), 'ReLu', (n, n), 'Sigmoid', (n, 1)], 'MSE', 'Adam', lr)
    image_path = model.image_path(prefix)
    saved_model_path = f"{prefix}/{model.id()}.pt"

    if Path(image_path).is_file():
        print(f"Model exists: {image_path}")
        del model
        return

    if Path(saved_model_path).is_file():
        print(f"Model is already saved: {saved_model_path}")
        del model
        return

    model.load(dataset, 0.05, 128)
    model.train(niter)
    model.validation(v1, v2, vrange)

    try:
        model.save_losses(prefix=prefix)
    except:
        time.sleep(10)
        try:
            model.save_losses(prefix=prefix)
        except:
            print(f"save_losses failed")

    try:
        torch.save(model.state_dict(), saved_model_path)
    except:
        time.sleep(10)
        try:
            torch.save(model.state_dict(), saved_model_path)
        except:
            print(f"model save failed")

    try:
        model.plot(y_range=(-8, -2), prefix=prefix)
    except:
        try:
            model.plot(y_range=(-8, -2), prefix=prefix)
        except:
            print(f"plotly failed")

    del model

# brute_force(
#     niter=128,
#     v1=lambda x: 10**(dp * x + dp),
#     v2=lambda x: 10**(dp * x + dp),
#     vrange=1e-6,
#     yrange=(-6, -1.5)
# )
# single_learning(niter=10240)

In [ ]:
def load_and_check(path):
    n = 128
    lr = 0.001
    
    try:
        model = RSModel([(5, n), 'ReLu', (n, n), 'Sigmoid', (n, 1)], 'MSE', 'Adam', lr)
        model.load_state_dict(torch.load(path))
        model.load(dataset, 0.05, 128)
    except:
        print(f"Cannot load the model! {path}")
        return
    
    v1=lambda x: 10**(dp * x + dp)
    v2=lambda x: 10**(dp * x + dp)
    vrange=0.05
    
    model.validation(v1, v2, vrange)
    
    return model


def compare_model_with_fortran(path):
    n = 128
    lr = 0.001
    inp = [-0.183694527, -0.293414205, -0.183694527, -0.293414205, 0.0]
    
    try:
        model = RSModel([(5, n), 'ReLu', (n, n), 'Sigmoid', (n, 1)], 'MSE', 'Adam', lr)
        model_params = torch.load(path)
        model.load_state_dict(model_params)
        model.load(dataset, 0.05, 128)
    except:
        print(f"Cannot load the model! {path}")
        return
    
    print('inaj')
    def relu(x):
        return np.maximum(x, 0)
    
    def sigmoid(x):
        return 1/(1 + np.exp(-x))
    
    w, b = [], []
    for i in range(3):
        wtag = f"model.L{i+1}.weight"
        btag = f"model.L{i+1}.bias"
        
        w.append(model_params[wtag].numpy().transpose())
        b.append(model_params[btag].numpy().transpose())
        
    print('w0', w[0][0, :5])
    print('b0', b[0][:5])
    print('w1', w[1][0, :5])
    print('b1', b[1][:5])
    print('w2', w[2][:15, 0])
    print('b2', b[2][:1])
    
    mod = model(torch.FloatTensor(inp))
    fort = np.matmul(sigmoid(np.matmul(relu(np.matmul(inp, w[0]) + b[0]), w[1]) + b[1]), w[2]) + b[2]
    
    print(mod.detach().numpy()[0], fort[0])

# m = load_and_check(
#     '/Users/Saeed/Desktop/single_long_run_models.norm.done/5_128-ReLu-128_128-Sigmoid-128_1-MSE-Adam-0_001.pt',
# )

compare_model_with_fortran(
    '/Users/Saeed/Desktop/single_long_run_models.norm.done/5_128-ReLu-128_128-Sigmoid-128_1-MSE-Adam-0_001.pt'
)

In [ ]:
def torch_to_rhyme(path):
    h5path = f"{path}.h5"
    
    if Path(h5path).is_file():
        print(f"File exists! {h5path}")
        return
        
    try:
        model = torch.load(path)
    except:
        print('Cannot load the model!')
        return
    
    f = h5py.File(h5path, "w")
    
    f.attrs['n_layers'] = int(len(model.keys()) / 2)
    
    f.attrs['model_unit_system'] = 'SI'
    
    f.attrs['d_norm'] = "(log10(rho_0) - drho) / drho"
    f.attrs['drho'] = drho
    
    f.attrs['p_norm'] = "(log10(p_0) - dp) / dp"
    f.attrs['dp'] = dp
    
    f.attrs['v_norm'] = "(v_r - v_l) / dv"
    f.attrs['dv'] = 2 * v_max.si.value
    
    w, b = [], []
    for i in range(3):
        model_params = torch.load(path)
        
        wtag = f"model.L{i+1}.weight"
        btag = f"model.L{i+1}.bias"
        
        w.append(np.asfortranarray(model_params[wtag].numpy().transpose()))
        b.append(np.asfortranarray(model_params[btag].numpy().transpose()))
    
    for i in range(f.attrs['n_layers']):
        lay = f.create_group(f'layer_{i}')
        
        lay.attrs[f'weights_shape'] = w[i].shape
        lay.attrs[f'biases_shape'] = b[i].shape
        print('w', w[i].shape)
        print('b', b[i].shape)
        
        wdata = lay.create_dataset('weights', w[i].shape, dtype=np.float32)
        wdata[:, :] = w[i]
        
        bdata = lay.create_dataset('biases', [1, b[i].shape[0]], dtype=np.float32)
        bdata[0, :] = b[i]
        
    f.close()
    
torch_to_rhyme('/Users/Saeed/Desktop/single_long_run_models.norm.done/5_128-ReLu-128_128-Sigmoid-128_1-MSE-Adam-0_001.pt')

### Tests <a class="anchor" id="mlrs-te"></a>

## Conclusion <a class="anchor" id="con"></a>

## References <a class="anchor" id="refer"></a>
* [A Finite Difference Method for the Computation of Discontinuous Solutions of the Equations of Fluid Dynamics](http://www.mathnet.ru/links/ece056bd10695ed71d90b68f97f15e20/sm4873.pdf) by S. K. Godunov [1959] <a class="anchor" id="ref-godunov-1959"></a>
* [Solution in the Large for Nonlinear Hyperbolic Systems of Equations](https://onlinelibrary.wiley.com/doi/epdf/10.1002/cpa.3160180408) by J. Glimm [1965] <a class="anchor" id="ref-glimm-1965"></a>
* [Efficient Solution Algorithms for the Riemann Problem for Real Gases](https://www.sciencedirect.com/science/article/pii/0021999185901469) by P. Colella and H. H. Glaz [1985] <a class="anchor" id="ref-colella-1985"></a>

## Similar works <a class="anchor" id="similar-works"></a>
* [Constraint-Aware Neural Networks for RiemannProblems](file:///Users/Saeed/Downloads/ML_Riemann_2019.pdf) by Magiera et al. [2019]
* [Machine learning approaches for the solution of the Riemann problem in fluid dynamics: a case study](file:///Users/Saeed/Downloads/ML_QE_RP.pdf) by Gyrya et al. [2019]

## Appendix I: Derivation of pressure function for shocks and rarefactions <a class="anchor" id="app-1"></a>

In order to derive the pressure function for shock waves we move to a frame of reference moving at the shock velocity and we introduce the relative velocities as,

$$\hat{u}_K = u_K - S_K, \quad \hat{u}_* = u_* - S_K$$

By substituting Rankine-Hugoniot conditions in to $f_K$ equation we can derive a relation for shock waves as,

$$
\begin{aligned}
  \rho_K \hat{u}_K &= \rho_* \hat{u}_* \\
  \rho_K \hat{u}_K^2 + p_K &= \rho_* \hat{u}_*^2 + p_* \\
  \hat{u}_K (\hat{E}_K + p_K) &= \hat{u}_* (\hat{E}_* + p_*)
\end{aligned}
$$

* [ ] Substitution?

In the case of a rarefaction wave, $f_K$ can be derived by assuming the isentropic relation

$$p = C \rho^{\gamma}, \quad C = p_K / \rho_K^{\gamma}$$

from which we write,

$$\rho_{*K} = \rho_K \left( \frac{p_*}{p_K} \right)^{\frac 1 \gamma}$$

and Generalised Riemann Invariant

$$I_K(u, c_s) = u_K + \frac{2 c_{sK}}{\gamma - 1}$$

which is constant across a rarefaction wave, so we can write,

$$
\begin{aligned}
  & u_L + \frac{2 c_{sL}}{\gamma - 1} = u_* + \frac{2 c_{s*L}}{\gamma - 1} \\
  & u_* - \frac{2 c_{s*R}}{\gamma - 1} = u_R - \frac{2 c_{sR}}{\gamma - 1}
\end{aligned}
$$

where,

$$
c_{s*K} = c_{sK} \left(
  \frac{p_*}{p_K}
\right)^{\frac{\gamma - 1}{2 \gamma}}
$$